In [1]:
import pandas as pd
import numpy as np


from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier

In [7]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

# Set up input data
AAP = 202
SGPT = 22
TB = 0.9
DB = 0.3
SGOT = 19
AGR = 1.2
ALBA = 4.1
AGE=65
GEN=1
TP=6.8


input = np.array([AGE,GEN,TB, DB, AAP, SGPT, SGOT,TP, ALBA, AGR])
columns = ['Age of the patient','Gender of the patient','Total Bilirubin', 'Direct Bilirubin', 'Alkphos Alkaline Phosphotase', 'Sgpt Alamine Aminotransferase', 'Sgot Aspartate Aminotransferase','Total Protiens', 'ALB Albumin', 'A/G Ratio Albumin and Globulin Ratio']
input_df = pd.DataFrame(input.reshape(1,-1), columns=columns)

# Load and preprocess training data
df=pd.read_csv(r"C:\Users\Sudeep Pawar\OneDrive\Documents\Project API\Liver Patient Dataset (LPD)_train.csv",encoding="unicode_escape")
from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()
df['Gender of the patient']=enc.fit_transform(df['Gender of the patient'])
df.info()
df['Result']=df['Result'].apply(lambda x:1 if x==2 else 0)

X=df.select_dtypes(exclude=object)
X=X.drop('Result',axis=1)
c_names=list(X.columns)
model=SimpleImputer(strategy='median')
model.fit(X)
X=model.transform(X)
X=pd.DataFrame(X,columns=c_names)
Y=df['Result']


param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
model = RandomForestClassifier()
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X, Y)

best_model = grid_search.best_estimator_
result = best_model.predict(input_df)
print(result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30691 entries, 0 to 30690
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Age of the patient                    30689 non-null  float64
 1   Gender of the patient                 30691 non-null  int32  
 2   Total Bilirubin                       30043 non-null  float64
 3   Direct Bilirubin                      30130 non-null  float64
 4   Alkphos Alkaline Phosphotase          29895 non-null  float64
 5   Sgpt Alamine Aminotransferase         30153 non-null  float64
 6   Sgot Aspartate Aminotransferase       30229 non-null  float64
 7   Total Protiens                        30228 non-null  float64
 8   ALB Albumin                           30197 non-null  float64
 9   A/G Ratio Albumin and Globulin Ratio  30132 non-null  float64
 10  Result                                30691 non-null  int64  
dtypes: float64(9), 

In [8]:
cv_results = grid_search.cv_results_
results_df = pd.DataFrame(cv_results)
results_df = results_df.sort_values(by='mean_test_score', ascending=False)
print((results_df[['params', 'mean_test_score']]).head(10))


                                               params  mean_test_score
5   {'max_depth': None, 'min_samples_leaf': 1, 'mi...         0.998371
1   {'max_depth': None, 'min_samples_leaf': 1, 'mi...         0.998273
4   {'max_depth': None, 'min_samples_leaf': 1, 'mi...         0.998273
3   {'max_depth': None, 'min_samples_leaf': 1, 'mi...         0.998208
8   {'max_depth': None, 'min_samples_leaf': 1, 'mi...         0.998175
2   {'max_depth': None, 'min_samples_leaf': 1, 'mi...         0.998143
6   {'max_depth': None, 'min_samples_leaf': 1, 'mi...         0.998078
11  {'max_depth': None, 'min_samples_leaf': 2, 'mi...         0.998045
14  {'max_depth': None, 'min_samples_leaf': 2, 'mi...         0.998045
13  {'max_depth': None, 'min_samples_leaf': 2, 'mi...         0.998012


In [9]:
grid_search.best_params_

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 150}

In [10]:
import joblib

In [11]:
joblib.dump(best_model, 'best_model_checkpoint.pkl')

['best_model_checkpoint.pkl']

In [12]:
model = joblib.load('best_model_checkpoint.pkl')
result = model.predict(input_df)
print(result)

[1]


In [4]:
import requests
import json

url = "http://127.0.0.1:5000/predict"

payload = json.dumps({
  "AAP": 202,
  "SGPT": 22,
  "TB": 0.9,
  "DB": 0.3,
  "SGOT": 19,
  "AGR": 1.2,
  "ALBA": 4.1
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


Patient has liver disease
